# PDF -> Embeddings -> Azure 

# Install Azure Cognitive Search SDK

In [1]:
# %pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004
# %pip install azure-identity
# %pip install langchain===0.0.200
# %pip install openai, tiktoken
# %pip install pypdf
# %pip install python-dotenv

In [2]:
%pip install azure-identity
%pip install langchain
%pip install openai, tiktoken
%pip install pypdf
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'openai,'


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install openai
!pip install tiktoken

## Import required libraries

In [4]:
############ RUN THIS ############

import os, json
import openai
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import BaseRetriever
from langchain.vectorstores.azuresearch import AzureSearch

## Configure OpenAI settings
Configure the OpenAI settings to use Azure OpenAI or OpenAI

In [5]:
############ RUN THIS ############

# Load environment variables from a .env file using load_dotenv():
load_dotenv()

azure_openai_api_key: str = os.environ.get('AZURE_OPENAI_API_KEY')

openai.api_type = "azure"
openai.api_base = "https://verx-corp-ai.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = azure_openai_api_key
openai.openai_api_key = azure_openai_api_key
model: str = "text-embedding-ada-002"

## Configure vector store settings
 
Set up the vector store settings using environment variables:
Change the index name to be your index...  the index name here will also be used for the similarity search below.

In [6]:
############ RUN THIS ############

vector_store_address: str = 'https://violet-vector.search.windows.net'
vector_store_password: str = os.environ.get('AZURE_VECTOR_STORE_PASSWORD')
index_name: str = "et-dossier-fuel-test"

## Create embeddings and vector store instances
 
Create instances of the OpenAIEmbeddings and AzureSearch classes:

In [7]:
############ RUN THIS ############

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=model, chunk_size=1, openai_api_key=azure_openai_api_key, deployment='VERX-CORP-ADA-002')  
vector_store: AzureSearch = AzureSearch(azure_search_endpoint=vector_store_address,  
                                        azure_search_key=vector_store_password,  
                                        index_name=index_name,  
                                        embedding_function=embeddings.embed_query)  



ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("../data/fuel-docs/")
docs = loader.load()
len(docs)

862

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000,chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)
len(split_docs)


859

In [ ]:
import time
import json
import requests
vjson = open('C:/Users/Divya.Koyyalamudi/Downloads/Azure Open AI Deployment.postman_collection 1.json')
json_data = json.load(vjson)
header_data = {json_data['item'][0]['request']['header'][0]['key'] : json_data['item'][0]['request']['header'][0]['value'] }
body_data = json.loads(json_data['item'][0]['request']['body']['raw'])
def generic_model(text, template,temperature, max_tokens ):
    """Generic gpt4 model
    """
  
    body_data['temperature'] = temperature
    body_data['max_tokens'] = max_tokens
    runner = None
    final_str = str()
    body_data['messages'][0]['content'] = template  + text   
    for i in range(0,1):
            body_data['messages'][0]['content'] = template + text      
            try:
                resp = requests.post('https://verx-corp-ai.openai.azure.com/openai/deployments/VERX-CORP-GPT40/chat/completions?api-version=2023-03-15-preview'
                        , headers = header_data, json = body_data)
                print(resp.status_code)
                #print(resp.json())
                final_str = resp.json()['choices'][0]['message']['content'] 
                
            except Exception as e:
                print('ERRORED OUT :(\n\n', e) 
                time.sleep(60) #35
                print('retrying...')
                resp = requests.post('https://verx-corp-ai.openai.azure.com/openai/deployments/VERX-CORP-GPT40/chat/completions?api-version=2023-03-15-preview'
                        , headers = header_data, json = body_data)
                print(resp.status_code)
                final_str = resp.json()['choices'][0]['message']['content']   
                
    print('Keywords generation successful!!')
    return final_str

In [ ]:
import tiktoken
encoding = tiktoken.encoding_for_model(model_name= 'gpt-4' )
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the no. of tokens in text string"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [ ]:
def output_max_tokens(text, template):
    """Input max_tokens
    """
    tokens_text = num_tokens_from_string(text,'cl100k_base')
    
   
    tokens_max_template = len(template) #len is biggest for a prompt considering that token
    print('tokens_text:' ,tokens_text,  'tokens_max_template:',tokens_max_template,)
    return int(32000- int(tokens_text) - int(tokens_max_template))

In [ ]:
output_final=str()
for i in range(0,1):
    text=text1
    template="Find the most frequently used keywords in the following text. Give the output in python list format"
    output=generic_model(text, template, temperature =0, max_tokens = output_max_tokens(text, template))
    output_final=output_final+ output
print(output_final)

tokens_text: 26488 tokens_max_template: 102


KeyboardInterrupt: 

In [ ]:
import os,openai
from dotenv import load_dotenv
# Load environment variables from a .env file using load_dotenv():
load_dotenv()

azure_openai_api_key: str = os.environ.get('AZURE_OPENAI_API_KEY')

openai.api_type = "azure"
openai.api_base = "https://verx-corp-ai.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = azure_openai_api_key
openai.openai_api_key = azure_openai_api_key
deployment_id: str = "VERX-CORP-DAVINCI"
model: str = "text-davinci-003"

from langchain import OpenAI 

from langchain import PromptTemplate, LLMChain
from langchain.llms import AzureOpenAI
from langchain.llms import AzureOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0, model=model, openai_api_key=azure_openai_api_key, deployment_id=deployment_id)



In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

#customize the prompt

map_prompt_template =  """You are a research analyst. I will provide you with a section of a document and you will create a list of most frquently used keywords.

Input: {text} """

combine_prompt_template = """You are a copy editor. Combine the below keywords to a single list. 

Input: {text} """

In [ ]:

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])
combine_prompt = PromptTemplate(template=combine_prompt_template, input_variables=["text"])
keyword=[]
chain = load_summarize_chain(llm, chain_type='map_reduce', map_prompt=map_prompt, combine_prompt=combine_prompt, verbose=False)
for i in range(0,len(split_docs),5):
    print(i)
    output_summary = chain.run(split_docs[i:i+5])
    wrapped_text = textwrap.fill(output_summary, width=80)
    keyword.append(wrapped_text)
    #print(wrapped_text)

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855


In [ ]:
Keywords_list=[''.join(keyword[0:len(keyword)])]
Keywords_array=str(Keywords_list[0]).split(",")
Keywords_set=set(Keywords_array)

In [ ]:
len(Keywords_set)

4989

In [ ]:
text1=Keywords_list[0]
print(text1)

 7. RAB 2018-14 8. 6% 9. statewide average retail price 10. gallon 11. undyed
No.2 ultra-low sulfur diesel fuel 12. rounded up 13. nearest 1/10 of 1 cent 14.
prepayment rates 15. gasoline 16. diesel fuel 17. effective 18. month of July
2018 19. determined 20. Department 21. self-serve unleaded regular gasoline 22.
May 1, 2018 23. May 31, 2018 24. Michigan Department of Treasury 25. Technical
Services Section 26. P.O. Box 30698 27. Lansing, MI 48909-8198 28. Telephone
(517) 636-4357.  Output: IFTA, Texas, Guidebook, International Fuel Tax Agreement, United
States, Canada, Tax Collection, Motor Fuel Taxes, Quarterly Tax Report, Fuel Tax
Audits, Articles of Agreement, Procedures Manual, Audit Manual, Application,
Licensing, Reporting, Recordkeeping, Audit Procedures, Non-IFTA Jurisdictions,
Alaska, District of Columbia, United Mexican States, Northwest Territories,
Nunavut Territory, Yukon Territory, Texas and Mexico, Interstate Trucker
License, Form AP-133, Fuel Trip Permit, Webfile, Cas

## Perform a vector similarity search
 
Execute a pure vector similarity search using the similarity_search() method:

In [ ]:
output=output_final.split(" ")
print(output)

['/nMost', 'Frequently', 'Used', 'Keywords:', '\n\n1.', 'Tax\n2.', 'Revenue\n3.', 'Department\n4.', 'Sales\n5.', 'Use', 'Tax\n6.', 'Motor', 'Fuel\n7.', 'Diesel\n8.', 'Exemptions\n9.', 'Administration\n10.', 'Rate\n11.', 'State\n12.', 'Local\n13.', 'Interest\n14.', 'Penalty\n15.', 'Act\n16.', 'Allocation\n17.', 'Collection\n18.', 'Payment\n19.', 'Return\n20.', 'Report\n21.', 'License\n22.', 'Property\n23.', 'Tribunal\n24.', 'Refund\n25.', 'Credit\n26.', 'Fuel\n27.', 'Gasoline\n28.', 'Taxpayer\n29.', 'Records\n30.', 'Agreement\n31.', 'Taxable\n32.', 'Exemption\n33.', 'Michigan\n34.', 'Wyoming\n35.', 'GHG\n36.', 'Inventory\n37.', 'CCS\n38.', 'Environmental', 'Quality\n39.', 'State', 'Treasurer\n40.', 'Department', 'of', 'Revenue\n41.', 'Sales', 'Tax\n42.', 'Use', 'Tax', 'Act\n43.', 'State', 'Tax', 'Commission\n44.', 'Administrative', 'Procedures', 'Act\n45.', 'Tax', 'Tribunal\n46.', 'Tax', 'Rate\n47.', 'Tax', 'Collection\n48.', 'Tax', 'Law\n49.', 'Taxable', 'Miles\n50.', 'Taxable', 'Gallo

In [ ]:
import re


kws = ['/nMost', 'Frequently', 'Used', 'Keywords:', '\n\n1.', 'Tax\n2.', 'Revenue\n3.', 'Department\n4.', 'Sales\n5.', 'Use', 'Tax\n6.', 'Motor', 'Fuel\n7.', 'Diesel\n8.', 'Exemptions\n9.', 'Administration\n10.', 'Rate\n11.', 'State\n12.', 'Local\n13.', 'Interest\n14.', 'Penalty\n15.', 'Act\n16.', 'Allocation\n17.', 'Collection\n18.', 'Payment\n19.', 'Return\n20.', 'Report\n21.', 'License\n22.', 'Property\n23.', 'Tribunal\n24.', 'Refund\n25.', 'Credit\n26.', 'Fuel\n27.', 'Gasoline\n28.', 'Taxpayer\n29.', 'Records\n30.', 'Agreement\n31.', 'Taxable\n32.', 'Exemption\n33.', 'Michigan\n34.', 'Wyoming\n35.', 'GHG\n36.', 'Inventory\n37.', 'CCS\n38.', 'Environmental', 'Quality\n39.', 'State', 'Treasurer\n40.', 'Department', 'of', 'Revenue\n41.', 'Sales', 'Tax\n42.', 'Use', 'Tax', 'Act\n43.', 'State', 'Tax', 'Commission\n44.', 'Administrative', 'Procedures', 'Act\n45.', 'Tax', 'Tribunal\n46.', 'Tax', 'Rate\n47.', 'Tax', 'Collection\n48.', 'Tax', 'Law\n49.', 'Taxable', 'Miles\n50.', 'Taxable', 'Gallons']

keywords_final= []
for word in kws:
    word = word.replace('\n', '')
    word = word.replace('/n', '')
    word = re.sub('\d+\.', '', word)
    keywords_final.append(word)

keywords_final=keywords_final[4:50]

In [ ]:
############ RUN THIS TO TEST FUNCTIONALITY ############
# Perform a similarity search
keyword_definitions=[]
print(len(keywords_final))
#for i in range(0,len(keywords_final)):
docs=[]
docs = vector_store.similarity_search(query=str(keywords_final[i]), k=1, search_type='similarity')
print(docs[0].page_content)
keyword_definitions.append(str(docs[0].page_content))
    
print(keyword_definitions)    
    


46


InvalidRequestError: [''] is not valid under any of the given schemas - 'input'

In [ ]:
############ START QUERYING HERE ############





